<a href="https://colab.research.google.com/github/Ctc4/ATV-BIGDATA-SPARK/blob/main/COMECANDO_SPARK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TRABALHANDO COM SPARK**

**Iniciando**

In [5]:
!pip install -q findspark

In [3]:
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.1 MB/s eta 0:00:00


In [40]:

import os

In [41]:

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [42]:
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

In [43]:
import findspark

In [44]:
findspark.init

<function findspark.init(spark_home=None, python_path=None, edit_rc=False, edit_profile=False)>

**Criando instância do PySpark** - estabelecer uma conexão com um cluster

In [45]:
from pyspark import SparkContext

In [46]:
spark_contexto = SparkContext()

ValueError: ignored

In [26]:
print(spark_contexto)

NameError: ignored

**Criando uma sessão do Spark**

In [12]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()    #create my_spark

In [13]:
print(spark)     #imprime o objeto spark

**Lendo os dados**

In [14]:
dataset = spark.read.csv('/content/sample_data/california_housing_test.csv',inferSchema=True, header =True)

In [15]:
dataset.head()

Row(longitude=-122.05, latitude=37.37, housing_median_age=27.0, total_rooms=3885.0, total_bedrooms=661.0, population=1537.0, households=606.0, median_income=6.6085, median_house_value=344700.0)

In [16]:
dataset.count()

3000

**Criando uma tabela SQL temporária com os dados do Dataset**

In [17]:
dataset.createOrReplaceTempView("tabela_temporaria")

In [18]:
print(spark.catalog.listTables()) 

[Table(name='tabela_temporaria', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


**Fazendo uma consulta SQL**

In [19]:
query = "FROM tabela_temporaria SELECT longitude, latitude LIMIT 3" 

In [20]:
saida = spark.sql(query)  

In [21]:
saida.show() 

+---------+--------+
|longitude|latitude|
+---------+--------+
|  -122.05|   37.37|
|   -118.3|   34.26|
|  -117.81|   33.78|
+---------+--------+



**Usando PySpark com Pandas**

In [27]:
query1 = "SELECT MAX(total_rooms) as maximo_quartos FROM tabela_temporaria"

In [28]:
q_maximo_quartos = spark.sql(query1)

In [29]:
pd_maximo_quartos = q_maximo_quartos.toPandas()

In [30]:
print('A quantidade máxima de quartos é: {}'.format(pd_maximo_quartos['maximo_quartos']))

A quantidade máxima de quartos é: 0    30450.0
Name: maximo_quartos, dtype: float64


In [31]:
qtd_maximo_quartos = int(pd_maximo_quartos.loc[0,'maximo_quartos'])

In [32]:
query2 = "SELECT longitude, latitude FROM tabela_temporaria WHERE total_rooms = "+str(qtd_maximo_quartos)

In [33]:
localizacao_maximo_quartos = spark.sql(query2)

In [34]:
pd_localizacao_maximo_quartos = localizacao_maximo_quartos.toPandas()

In [35]:
print(pd_localizacao_maximo_quartos.head()) 

   longitude  latitude
0     -117.2     33.58


**Convertendo Pandas DataFrame para Spark DataFrame**

In [39]:
import pandas as pd
import numpy as np
media = 0
desvio_padrao=0.1 
pd_temporario = pd.DataFrame(np.random.normal(media,desvio_padrao,100))
spark_temporario = spark.createDataFrame(pd_temporario)

In [37]:
print(spark.catalog.listTables())

[Table(name='nova_tabela_temporaria', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='tabela_temporaria', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [38]:
spark_temporario.createOrReplaceTempView("nova_tabela_temporaria")
print(spark.catalog.listTables())   

[Table(name='nova_tabela_temporaria', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='tabela_temporaria', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]
